# Tiền xử lí

In [1]:
import cv2
import os

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

In [ ]:
def extract_faces_from_image(image_path, output_path, face_count):
    print(f"Processing image: {image_path}")
    img = cv2.imread(image_path)
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in faces:
        face = cv2.resize(img[y+1:y+h-1, x+1:x+w-1], (128, 128))
        face_count += 1
        cv2.imwrite(f'{output_path}/face_{face_count}.jpg', face)
    return face_count

In [ ]:
def extract_faces_from_video(video_path, output_path, face_count):
    print(f"Processing video: {video_path}")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Can't open {video_path}")
        return face_count

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces:
            roi = cv2.resize(frame[y+1:y+h-1, x+1:x+w-1], (128, 128))
            face_count += 1
            cv2.imwrite(f'{output_path}/face_{face_count}.jpg', roi)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return face_count

In [ ]:
def process_data_directory(data_path):
    for category in os.listdir(data_path):
        if category != '.DS_Store':
            category_path = os.path.join(data_path, category)
            face_count = 0
            output_path = category_path.replace('dataRaw', 'dataset')
            
            os.makedirs(output_path, exist_ok=True)

            for data_item in os.listdir(category_path):
                if data_item != '.DS_Store':
                    item_path = os.path.join(category_path, data_item)

                    if item_path.endswith('.mp4'):
                        face_count = extract_faces_from_video(item_path, output_path, face_count)

                    if item_path.endswith(('.jpg', '.png', '.jpeg')):
                        face_count = extract_faces_from_image(item_path, output_path, face_count)

            print(f"Total faces in {category}: {face_count}")

In [ ]:
# Process training and testing data directories
data_train_path = 'dataRaw/train_data'
data_test_path = 'dataRaw/test_data'

process_data_directory(data_train_path)
process_data_directory(data_test_path)

Processing video: dataRaw/train_data/Justin Bieber/justinbieber.mp4
Total faces in Justin Bieber: 1694
Processing video: dataRaw/train_data/David Beckham/davidbekham.mp4
Total faces in David Beckham: 878
Processing image: dataRaw/train_data/Reus/image.jpg
Processing video: dataRaw/train_data/Reus/reus.mp4
Total faces in Reus: 913
Processing video: dataRaw/train_data/Ronaldo/ronaldo.mp4
Total faces in Ronaldo: 2233
Processing video: dataRaw/train_data/Messi/messi.mp4
Total faces in Messi: 1638
Processing video: dataRaw/train_data/Son Tung/sontung.mp4
Total faces in Son Tung: 1298
Processing video: dataRaw/train_data/G-Dragon/GD.mp4
Total faces in G-Dragon: 1743
Processing image: dataRaw/test_data/Justin Bieber/Justin-Bieber-0.jpg
Processing image: dataRaw/test_data/Justin Bieber/images (13).jpg
Processing image: dataRaw/test_data/Justin Bieber/jbasdf.jpg
Processing image: dataRaw/test_data/Justin Bieber/Ca-Si-Justin-Bieber.jpg
Processing image: dataRaw/test_data/Justin Bieber/justin-bie

# Huấn luyện mô hình

In [2]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from PIL import Image

2023-12-02 18:04:33.571359: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 18:04:33.725566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 18:04:33.726366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 18:04:34.668130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
TRAIN_DATA = 'dataset/train_data'
TEST_DATA = 'dataset/test_data'
print(tf.__version__)

2.12.0


In [4]:
classes = []

# Lists to store data for training and testing
xtrain = []
ytrain = []
xtest = []
ytest = []

# Dictionary to map class names to labels
dictionary = {}
np.random.seed(5)

In [5]:
#Get Dictionary from Data
class Dictionary:
    def __init__(self, path):
        self.path = path
        self.dictionary = {}
    def operate(self):
        number = 0
        for name in os.listdir(self.path):
            if not name.startswith('.'):
                self.dictionary[name] = number
                number += 1
        return self.dictionary

In [6]:
# Example debugging in the Data class
class Data:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.input = []
        self.label = []
        self.listData = []

    def getData(self):
        for whatever in os.listdir(self.path):
            if whatever != '.DS_Store':
                label = whatever
                whatever_path = os.path.join(self.path, whatever)
                list_data_filename_path = []
                
                for filename in os.listdir(whatever_path):
                    if filename != '.DS_Store':
                        filename_path = os.path.join(whatever_path, filename)
                        if filename_path.endswith(('.jpg', '.png', '.jpeg')):
                            img = np.array(Image.open(filename_path))
                            list_data_filename_path.append((img, self.dictionary[label]))

                self.listData.extend(list_data_filename_path)
        return self.listData

    def operate(self):
        self.listData = self.getData()
        np.random.shuffle(self.listData)
        for Data in self.listData:
            self.input.append(np.array(Data[0])) 
            self.label.append(Data[1])
        return self.input, self.label

In [9]:
dictionary = Dictionary(TRAIN_DATA).operate()
classes = list(dictionary.keys())
print(dictionary)

{'Justin Bieber': 0, 'David Beckham': 1, 'Reus': 2, 'Ronaldo': 3, 'Messi': 4, 'Son Tung': 5, 'G-Dragon': 6}


In [10]:
print(os.listdir('dataset/train_data'))

['Justin Bieber', 'David Beckham', 'Reus', 'Ronaldo', 'Messi', 'Son Tung', 'G-Dragon']


In [2]:
# Example usage with debugging
xtrain, list_train = Data(TRAIN_DATA, dictionary).operate()
xtest, list_test = Data(TEST_DATA, dictionary).operate()

NameError: name 'Data' is not defined

In [1]:
print(len(xtrain))
print(len(xtest))

NameError: name 'xtrain' is not defined

In [13]:
# Normalize data to the range [0, 1]
xtrain, xtest = np.array(xtrain) / 100, np.array(xtest) / 100

# Expand dimensions for the channel (3 for RGB)
ytrain, ytest = np.array([np.expand_dims(item, axis=0) for item in list_train]), np.array([np.expand_dims(item, axis=0) for item in list_test])

# Convert labels to one-hot encoding
ytrain_ohe, ytest_ohe = to_categorical(ytrain), to_categorical(ytest)

: 

In [ ]:
def build_cnn_model(input_shape=(128,128,3)):
    model = models.Sequential([
        layers.Conv2D(32,(3,3), input_shape=(128,128,3),activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Dropout(0.15),

        layers.Conv2D(64,(3,3),activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Dropout(0.2),

        layers.Conv2D(128,(3,3),activation='relu'),
        layers.MaxPool2D((2,2)),
        layers.Dropout(0.2),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(len(classes), activation='softmax')
    ])
    return model

In [ ]:
def build_siamese_model(input_shape=(128,128,3)):
    input_a = tf.keras.Input(shape=input_shape)
    input_b = tf.keras.Input(shape=input_shape)

    base_model = build_cnn_model(input_shape)

    processed_a = base_model(input_a)
    processed_b = base_model(input_b)

    distance = tf.keras.layers.Lambda(lambda x: tf.abs(x[0] - x[1]))([processed_a, processed_b])
    output = layers.Dense(1, activation='sigmoid')(distance)

    model = tf.keras.Model(inputs=[input_a, input_b], outputs=output)
    return model

In [ ]:
siamese_model = build_siamese_model()

In [ ]:
siamese_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
history = siamese_model.fit(xtrain,ytrain_ohe,epochs=10,validation_data=(xtest,ytest_ohe))

NameError: name 'model_training' is not defined

In [ ]:
#Vẽ biểu đồ của accuracy và val_accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
test_loss, test_acc = siamese_model.evaluate(xtest, ytest_ohe, verbose=2)
plt.show()

# Test

In [ ]:
xtrain, list_train = Data(TRAIN_DATA, dictionary).operate()
xtest, list_test = Data(TEST_DATA, dictionary).operate()
xtrain, xtest = np.array(xtrain)/100, np.array(xtest)/100
ytrain, ytest = np.array([np.expand_dims(item, axis=0) for item in list_train]), np.array([np.expand_dims(item, axis=0) for item in list_test])
classes = list(dictionary.keys())

In [ ]:
acc = 0
fig = plt.figure(figsize=(15,15))
for i in range(50):
    plt.subplot(5,10,i+1)
    #Vì dataset/train chưa đủ lớn, đa dạng nên để xtest thì tỷ lệ đúng ít, thử thay xtest, ytest thành xtrain, ytrain (tỉ lệ đúng khá cao)
    plt.imshow(xtrain[i])
    result = np.argmax(siamese_model.predict(xtrain[i].reshape((-1,128,128,3))))
    if result == ytrain[i][0]:
        acc +=1
    plt.title(classes[result])
    plt.grid(color='lightgray', linestyle='--')
    plt.axis("off")

print(acc)
plt.show()